# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 3 2022 10:07AM,238743,10,BPS1244,Beach Patient Assistance,Released
1,Jun 3 2022 10:07AM,238743,10,BPS1245,Beach Patient Assistance,Released
2,Jun 3 2022 10:07AM,238743,10,BPS1242,Beach Patient Assistance,Released
3,Jun 3 2022 10:07AM,238742,10,8099537,Beach Products Inc,Released
4,Jun 3 2022 10:06AM,238741,10,BPS1243,Beach Patient Assistance,Released
5,Jun 3 2022 10:02AM,238721,18,IF-2091178,"Blinc International, Inc. - PVecom",Released
6,Jun 3 2022 9:54AM,238734,10,Enova-10069,Emerginnova,Released
7,Jun 3 2022 9:54AM,238734,10,Enova-10070,Emerginnova,Released
8,Jun 3 2022 9:43AM,238732,20,R1-8100042,Reliable 1 Laboratories LLC,Released
9,Jun 3 2022 9:41AM,238731,20,ALU0006972,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,238732,Released,1
34,238734,Released,2
35,238741,Released,1
36,238742,Released,1
37,238743,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238732,NaN,NaN,1.0
238734,NaN,NaN,2.0
238741,NaN,NaN,1.0
238742,NaN,NaN,1.0
238743,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238643,0.0,3.0,14.0
238644,0.0,1.0,0.0
238647,0.0,0.0,1.0
238649,15.0,8.0,1.0
238650,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238643,0,3,14
238644,0,1,0
238647,0,0,1
238649,15,8,1
238650,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238643,0,3,14
1,238644,0,1,0
2,238647,0,0,1
3,238649,15,8,1
4,238650,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238643,,3,14
1,238644,,1,
2,238647,,,1
3,238649,15,8,1
4,238650,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 3 2022 10:07AM,238743,10,Beach Patient Assistance
3,Jun 3 2022 10:07AM,238742,10,Beach Products Inc
4,Jun 3 2022 10:06AM,238741,10,Beach Patient Assistance
5,Jun 3 2022 10:02AM,238721,18,"Blinc International, Inc. - PVecom"
6,Jun 3 2022 9:54AM,238734,10,Emerginnova
8,Jun 3 2022 9:43AM,238732,20,Reliable 1 Laboratories LLC
9,Jun 3 2022 9:41AM,238731,20,Alumier Labs Inc.
27,Jun 3 2022 9:23AM,238730,12,Hush Hush
28,Jun 3 2022 9:23AM,238729,12,Hush Hush
37,Jun 3 2022 9:10AM,238727,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 3 2022 10:07AM,238743,10,Beach Patient Assistance,,,3
1,Jun 3 2022 10:07AM,238742,10,Beach Products Inc,,,1
2,Jun 3 2022 10:06AM,238741,10,Beach Patient Assistance,,,1
3,Jun 3 2022 10:02AM,238721,18,"Blinc International, Inc. - PVecom",,,1
4,Jun 3 2022 9:54AM,238734,10,Emerginnova,,,2
5,Jun 3 2022 9:43AM,238732,20,Reliable 1 Laboratories LLC,,,1
6,Jun 3 2022 9:41AM,238731,20,Alumier Labs Inc.,,,18
7,Jun 3 2022 9:23AM,238730,12,Hush Hush,,,1
8,Jun 3 2022 9:23AM,238729,12,Hush Hush,,,9
9,Jun 3 2022 9:10AM,238727,10,ISDIN Corporation,,,32


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 10:07AM,238743,10,Beach Patient Assistance,3,,
1,Jun 3 2022 10:07AM,238742,10,Beach Products Inc,1,,
2,Jun 3 2022 10:06AM,238741,10,Beach Patient Assistance,1,,
3,Jun 3 2022 10:02AM,238721,18,"Blinc International, Inc. - PVecom",1,,
4,Jun 3 2022 9:54AM,238734,10,Emerginnova,2,,
5,Jun 3 2022 9:43AM,238732,20,Reliable 1 Laboratories LLC,1,,
6,Jun 3 2022 9:41AM,238731,20,Alumier Labs Inc.,18,,
7,Jun 3 2022 9:23AM,238730,12,Hush Hush,1,,
8,Jun 3 2022 9:23AM,238729,12,Hush Hush,9,,
9,Jun 3 2022 9:10AM,238727,10,ISDIN Corporation,32,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 10:07AM,238743,10,Beach Patient Assistance,3,,
1,Jun 3 2022 10:07AM,238742,10,Beach Products Inc,1,,
2,Jun 3 2022 10:06AM,238741,10,Beach Patient Assistance,1,,
3,Jun 3 2022 10:02AM,238721,18,"Blinc International, Inc. - PVecom",1,,
4,Jun 3 2022 9:54AM,238734,10,Emerginnova,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 10:07AM,238743,10,Beach Patient Assistance,3.0,NaN,NaN
1,Jun 3 2022 10:07AM,238742,10,Beach Products Inc,1.0,NaN,NaN
2,Jun 3 2022 10:06AM,238741,10,Beach Patient Assistance,1.0,NaN,NaN
3,Jun 3 2022 10:02AM,238721,18,"Blinc International, Inc. - PVecom",1.0,NaN,NaN
4,Jun 3 2022 9:54AM,238734,10,Emerginnova,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 10:07AM,238743,10,Beach Patient Assistance,3.0,0.0,0.0
1,Jun 3 2022 10:07AM,238742,10,Beach Products Inc,1.0,0.0,0.0
2,Jun 3 2022 10:06AM,238741,10,Beach Patient Assistance,1.0,0.0,0.0
3,Jun 3 2022 10:02AM,238721,18,"Blinc International, Inc. - PVecom",1.0,0.0,0.0
4,Jun 3 2022 9:54AM,238734,10,Emerginnova,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4296639,157.0,19.0,22.0
12,716176,26.0,0.0,0.0
16,477298,1.0,1.0,0.0
18,238721,1.0,0.0,0.0
19,477368,3.0,0.0,0.0
20,716145,20.0,19.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4296639,157.0,19.0,22.0
1,12,716176,26.0,0.0,0.0
2,16,477298,1.0,1.0,0.0
3,18,238721,1.0,0.0,0.0
4,19,477368,3.0,0.0,0.0
5,20,716145,20.0,19.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,157.0,19.0,22.0
1,12,26.0,0.0,0.0
2,16,1.0,1.0,0.0
3,18,1.0,0.0,0.0
4,19,3.0,0.0,0.0
5,20,20.0,19.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,157.0
1,12,Released,26.0
2,16,Released,1.0
3,18,Released,1.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20
Status,,,,,,
Completed,22.0,0.0,0.0,0.0,0.0,0.0
Executing,19.0,0.0,1.0,0.0,0.0,19.0
Released,157.0,26.0,1.0,1.0,3.0,20.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20
0,Completed,22.0,0.0,0.0,0.0,0.0,0.0
1,Executing,19.0,0.0,1.0,0.0,0.0,19.0
2,Released,157.0,26.0,1.0,1.0,3.0,20.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20
0,Completed,22.0,0.0,0.0,0.0,0.0,0.0
1,Executing,19.0,0.0,1.0,0.0,0.0,19.0
2,Released,157.0,26.0,1.0,1.0,3.0,20.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()